In [ ]:
import pandas as pd
import os
import numpy as np
import json
from statsmodels.sandbox.stats.multicomp import multipletests as mt

In [ ]:
# get local dropbox folder path since all the data is in my dropbox
dropbox_path_file = ''.join(['c:/Users/',os.getlogin(),'/AppData/Local/Dropbox/info.json'])
try:
    with open(dropbox_path_file) as data_file:    
        data = json.load(data_file)
except:
    with open('C:/Users/Surbhi/AppData/Local/Dropbox/info.json') as data_file:    
        data = json.load(data_file)
dropbox_path = data['personal']['path']
os.chdir(dropbox_path)
Lincs_geneinfo = pd.read_table('Ontology_Info//GSE92742_Broad_LINCS_gene_info.txt',index_col = 0)
Geneidtable = pd.read_table('Ontology_Info/HGNC protein-coding_gene.txt',dtype = str,index_col = 0).symbol.tolist()

In [ ]:
# compiling clue query signatures from DE genes
os.chdir('E:/Box Sync/Jake-Jegga/IPF Drug Discovery/Datasets')
# processing differentially expressed genes from R limma output
fc_t = 0.6
files = os.listdir()
deg_files = [x for x in files if ('report.txt' in x) & ('noFDR' not in x) & ('GSE32537' not in x)]
deg = pd.DataFrame(index = list(range(20000)))
for fn in deg_files:    
    gse_id = fn.split(' ')[0]
    df = pd.read_table(fn,index_col = 0)
    up_genes = list(set(df[df.ix[:,0]>=fc_t].sort_values(df.columns[0],ascending = False).index.dropna()))
    dn_genes = list(set(df[df.ix[:,0]<=-fc_t].sort_values(df.columns[0]).index.dropna()))
    for reg,gl in zip(['_Up','_Dn'],[up_genes,dn_genes]):
        deg.ix[list(range(len(gl))),gse_id+reg] = gl
deg = deg.dropna(how='all')
deg.to_excel('../Intermediate results/Deg lists included in paper/All FC06 pval005 protein coding DEG.xlsx')
# making network ready file for visualizing in cytoscape
gene_statistics = deg.stack()
gene_statistics = gene_statistics.reset_index()
network = pd.DataFrame(gene_statistics.set_index(0)['level_1'])
gene_statistics['reg'] = [x.split('_')[1] for x in gene_statistics.level_1]
gene_statistics = gene_statistics.groupby(0).reg.value_counts()
up_reg_genes = gene_statistics.loc[gene_statistics.index.get_level_values('reg') == 'Up'].unstack()
dn_reg_genes = gene_statistics.loc[gene_statistics.index.get_level_values('reg') == 'Dn'].unstack()
gene_statistics = pd.concat([up_reg_genes,dn_reg_genes],axis =1).fillna(0)
gene_statistics.columns = ['# of up_reg_datasets','# of dn_reg_datasets']
# gene_statistics.to_excel('All FC1 pval005 protein coding DEG summary statistics.xlsx')
network.columns = ['dataset']
network['regulation'] = [x.split('_')[1] for x in network.dataset]
network = network.join(gene_statistics,how='outer')
network['# of nondeg datasets'] = len(deg_files) - network.ix[:,2] - network.ix[:,3]
network.to_excel('../Intermediate results/Deg lists included in paper/All FC06 pval005 protein coding DEG network.xlsx')
# make clue query file
deg = deg.apply(lambda x: [x for x in x.values if x in Lincs_geneinfo.ix[:,0].values][:150],axis=0)
clue_query = pd.DataFrame(index = list(range(150)),columns=deg.index)
for i in deg.index:
    clue_query.iloc[:len(deg[i])].loc[:,i] = deg[i]
clue_query.to_excel('../Intermediate results/Clue query/Clue_query FC06.xlsx')

In [ ]:
# calculate significance of disease-drug connectivity score by permutation
os.chdir('E:/Box Sync/Jake-jegga/IPF Drug Discovery/Intermediate results/Clue query')
fn = 'Combined report from 6 IPF mocroarry queries.gct'
clue = pd.read_table(fn,index_col=0)
clue.set_index('name',inplace=True)
clue = clue.iloc[:,5:]
clue = clue[[x for x in clue.columns if 'summary' not in x]]
cmpds = clue_report.index
connected = (clue>=90).sum(axis = 1).groupby(clue.index).max()
anti_connected = (clue<=-90).sum(axis = 1).groupby(clue.index).max()
clue_report = (connected-anti_connected)[cmpds]
i=0
pvalue_upper = pd.Series(0,index = cmpds)
while i<1000000:
    np.random.shuffle(clue.index.values)
    connected = (clue>=90).sum(axis = 1).groupby(clue.index).max()
    anti_connected = (clue<=-90).sum(axis = 1).groupby(clue.index).max()
    report = (connected-anti_connected)[cmpds]
    pvalue_upper += clue_report<=report
    if i%1000 == 0:
        print('Iteration {}'.format(i))
    i+=1
pval = (pvalue_upper/1000000).sort_values()
pval = pval[pval<=0.05]